# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# import numpy and pandas
import pandas as pd
import numpy as np
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
from statsmodels.stats.weightstats import ztest as ztest

from scipy.stats import ttest_ind, norm, t
from scipy.stats import f_oneway
from scipy.stats import sem

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [4]:

 salaries= pd.read_csv('Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [3]:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [5]:
missing_data = salaries.isnull().sum()
print("Missing Data in Each Column:")
print(missing_data)

Missing Data in Each Column:
Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64


Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [7]:
salary_hourly_count = salaries['Salary or Hourly'].value_counts()
print(salary_hourly_count)

Salary or Hourly
Salary    25161
Hourly     8022
Name: count, dtype: int64


What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [8]:
department_count = salaries['Department'].value_counts()
print(department_count)

Department
POLICE                   13414
FIRE                      4641
STREETS & SAN             2198
OEMC                      2102
WATER MGMNT               1879
AVIATION                  1629
TRANSPORTN                1140
PUBLIC LIBRARY            1015
GENERAL SERVICES           980
FAMILY & SUPPORT           615
FINANCE                    560
HEALTH                     488
CITY COUNCIL               411
LAW                        407
BUILDINGS                  269
COMMUNITY DEVELOPMENT      207
BUSINESS AFFAIRS           171
COPA                       116
BOARD OF ELECTION          107
DoIT                        99
PROCUREMENT                 92
INSPECTOR GEN               87
MAYOR'S OFFICE              85
CITY CLERK                  84
ANIMAL CONTRL               81
HUMAN RESOURCES             79
CULTURAL AFFAIRS            65
BUDGET & MGMT               46
ADMIN HEARNG                39
DISABILITIES                28
TREASURER                   22
HUMAN RELATIONS             

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [9]:
from statsmodels.stats.weightstats import ztest
# Since we have an N of 8022 and only one sample (Hourly workers), we go for a Z-test
# H0: Hourly rate == 30
# H1: Hourly rate != 30
salaries_hourly = salaries[salaries['Salary or Hourly'] == 'Hourly']


salaries_hourly = salaries[salaries['Salary or Hourly'] == 'Hourly']

z_stat, p_value = ztest(salaries_hourly['Hourly Rate'], value=30)

print(f"Z-statistic: {z_stat}")
print(f"P-value: {p_value}")

Z-statistic: 20.6198057854942
P-value: 1.8228873859286195e-94


We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [10]:
# Subset of the dataset: only Police department
salaries_police = salaries[(salaries['Department'] == 'POLICE') & (salaries['Salary or Hourly'] == 'Salary')]

In [11]:
# Since we have an N of 13404 and only one sample (salary of the police), we go for a Z-test
# H0: Annual Salary == 86000
# H1: Annual Salary > 86000

salaries_police = salaries[(salaries['Department'] == 'POLICE') & (salaries['Salary or Hourly'] == 'Salary')]

z_stat, p_value_two_tailed = ztest(salaries_police['Annual Salary'], value=86000)

p_value_one_tailed = p_value_two_tailed / 2

# Print the results
print(f"Z-statistic: {z_stat}")
print(f"One-tailed P-value: {p_value_one_tailed}")

if z_stat > 0 and p_value_one_tailed < 0.05:
    print("Reject the null hypothesis: The salaries this year are significantly higher than $86000.")
else:
    print("Fail to reject the null hypothesis: The salaries this year are not significantly higher than $86000.")

Z-statistic: 3.081997005712994
One-tailed P-value: 0.0010280845287547855
Reject the null hypothesis: The salaries this year are significantly higher than $86000.


Using the `crosstab` function, find the department that has the most hourly workers.

In [13]:
hourly_workers = salaries[salaries['Salary or Hourly'] == 'Hourly']

hourly_workers_count = pd.crosstab(hourly_workers['Department'], hourly_workers['Salary or Hourly'])

most_hourly_workers_dept = hourly_workers_count.idxmax().values[0]
most_hourly_workers_count = hourly_workers_count.max().values[0]

print(f"The department with the most hourly workers is {most_hourly_workers_dept} with {most_hourly_workers_count} hourly workers.")

The department with the most hourly workers is STREETS & SAN with 1862 hourly workers.


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [16]:
from scipy.stats import ttest_1samp

hourly_workers = salaries[salaries['Salary or Hourly'] == 'Hourly']

hourly_workers_count = pd.crosstab(hourly_workers['Department'], hourly_workers['Salary or Hourly'])

most_hourly_workers_dept = hourly_workers_count.idxmax().values[0]

most_hourly_workers = hourly_workers[hourly_workers['Department'] == most_hourly_workers_dept]

t_stat, p_value_two_tailed = ttest_1samp(most_hourly_workers['Hourly Rate'], 35)

p_value_one_tailed = p_value_two_tailed / 2

print(f"T-statistic: {t_stat}")
print(f"One-tailed P-value: {p_value_one_tailed}")

if t_stat < 0 and p_value_one_tailed < 0.05:
    print("Reject the null hypothesis: The hourly wage is significantly less than $35/hour.")
else:
    print("Fail to reject the null hypothesis: The hourly wage is not significantly less than $35/hour.")

T-statistic: -9.567447887848152
One-tailed P-value: 1.6689265282353859e-21
Reject the null hypothesis: The hourly wage is significantly less than $35/hour.
